In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Création du corpus
Le corpus est une collection de livres/Bande dessinés pour enfants

In [2]:
corpus=["Martine à la plage", "Martine en vacances","Martine en confinement","Martine chez ses amis","Martine à la décharge","Martine fait trempette", "Martine et les chiens", "Martine a peur","Martine à la ferme","Luky Luke chez les Chti's","Luky Lucke et Ma Dalton","Luky Luke et la Ruée vers l'or"]
print(corpus)

['Martine à la plage', 'Martine en vacances', 'Martine en confinement', 'Martine chez ses amis', 'Martine à la décharge', 'Martine fait trempette', 'Martine et les chiens', 'Martine a peur', 'Martine à la ferme', "Luky Luke chez les Chti's", 'Luky Lucke et Ma Dalton', "Luky Luke et la Ruée vers l'or"]


# Création de la matrice Tfidf
On va creer la matrice tfidf du corpus

In [3]:
vectorizer= TfidfVectorizer() # on laisse les options par défauts
# on va faire simple
matrice_tfidf=vectorizer.fit_transform(corpus)
vectorizer.get_feature_names()

['amis',
 'chez',
 'chiens',
 'chti',
 'confinement',
 'dalton',
 'décharge',
 'en',
 'et',
 'fait',
 'ferme',
 'la',
 'les',
 'lucke',
 'luke',
 'luky',
 'ma',
 'martine',
 'or',
 'peur',
 'plage',
 'ruée',
 'ses',
 'trempette',
 'vacances',
 'vers']

Mais que se passe t'il concrètement ?
Le texte subit d'abord un preprocessing puis une tokenization.
Grace à l'API de sklearn on peut récupérer ces 2 fonctions

In [4]:
preprocessor=vectorizer.build_preprocessor()
tokenizer=vectorizer.build_tokenizer()
test_preprocess=preprocessor("Toto fait de la cuisine")
test_preprocess

'toto fait de la cuisine'

Le preprocessing remet le texte en minuscule

In [5]:
tokenizer('toto fait de la cuisine')

['toto', 'fait', 'de', 'la', 'cuisine']

Le tokenizer découpe le texte en token ou jeton en français
Ces jetons sont stockés dans le vocabulaire.
Pour obtenir la matrice tfdif on décompose le texte en "sac de mot", on projète les mots sur le vocabulaire.

Prenons pour exemple le document 'Martine va la plage'

In [6]:
import pandas as pd
vecteur=vectorizer.transform(['martine va à la plage'])
df=pd.DataFrame(vecteur.toarray(),columns=vectorizer.get_feature_names())
df

,amis,chez,chiens,chti,confinement,dalton,décharge,en,et,fait,...,ma,martine,or,peur,plage,ruée,ses,trempette,vacances,vers
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.341494,0.0,0.0,0.776877,0.0,0.0,0.0,0.0,0.0


# Comment exploiter la matrice Tfidf dans le cadre de la recherche d'information
L'utilisateur formule une requête. Cette requête est tokenizé puis projeter sur le corpus.
En considérant les documents du corpus et la requête comme des vecteurs (Dont les coordonnées serait les coefficients de la matrice), on peut calculer le cosinus de l'angle entre les documents du corpus et la requête.
Le document recherché est celui qui possède le plus grand cosinus est le plus grand (c'est à dire une angle petit entre le document et la requête).

In [7]:
def requete(string):
    req=vectorizer.transform([string])
    req=req.transpose()
    long_r=req.shape[0]
    resultat=[]
    for indice,value in enumerate(corpus):
        cosinus= (matrice_tfidf.getrow(indice) @ req) # le cosinus est le produit scalaire divisé par le produit des normes
        resultat.append((indice,cosinus)) # ici sci-kit se charge de renormaliser les vecteurs pour nous
    resultat.sort(key=lambda item:item[1],reverse=True)
    indice,score=resultat[0]
    return(corpus[indice])

In [8]:
requete("Martine plage"), requete('Lucky Luke Dalton')

('Martine à la plage', 'Luky Lucke et Ma Dalton')

In [10]:
requete("Martine en vacances à la plage"),requete("Martine chez les Chti")

('Martine en vacances', "Luky Luke chez les Chti's")